## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [4]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./

--2024-01-19 17:45:44--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6025:18::a27d:4512, 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6025:18::a27d:4512|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2024-01-19 17:45:45--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc63453e6cc2769400c52675b8b6.dl.dropboxusercontent.com/cd/0/inline/CLrtHeF36RYkHACEgPpkK2ooPG8T0ZG3RVtCHqtTMhofoWSI1A290I9elTav1hWbViKIeAHacO4RFRXIYL6TQAbCH-nrk5T-oLsmfq6aj6288C7KQUSw5b-xODuROrqyaLQ/file# [following]
--2024-01-19 17:45:45--  https://uc63453e6cc2769400c52675b8b6.dl.dropboxusercontent.com/cd/0/inline/CLrtHeF36RYkHACEgPpkK2ooPG8T0ZG3RVtCHqtTMhofoWSI1A290I9elTav1hWbViKIeAHacO4RFRXIYL6TQAbCH-nrk5T

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Exercise 1
Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger than 10k.

In [6]:
# input file is in csv format, hence:
df = pd.read_csv('data_000637.txt')
df.describe()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 29, saw 2


Notes: TDC_MEAS has 30 bins, they cover a range of 25 nanoseconds. If I count 10, the time is 10*25/30 ns 
count from 0 to 29 and then goes back to 0, resets. When it resets the next counter gets updated.
BX_COUNTER is the next unit. Need to find the value 'x' at which BX_COUNTER gets reset.
The next after BX_COUNTER is the ORBIT_CNT.

So BX_counter is multiple of 25 ns, since every 25 ns the TDC_MEAS goes to 0.

the device has several channels, and the channels are marked by FPGA and TDC_CHANNEL.

#### Exercise 2
Find out the number of BX in a ORBIT (the value 'x').

In [ ]:
things = df.groupby('ORBIT_CNT')['BX_COUNTER'].max()
things.hist(bins=30)
things.describe()

In [ ]:
# the requested BX value that resets BX and makes the orbit go up by 1 will be the highest BX_COUNTER found, so:
bx_max = things.max()+1
print(f"BX 'x' value is {bx_max}")

#### Exercise 3
Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [ ]:
'''
so, the initial value is given by the number of orbits + bx + the fine measurement of the tdc. So:
'''
def ns_time(df, bx_max):
    t = {'ORBIT':0,'BX':0,'TDC':0}
    t['TDC'] = df['TDC_MEAS']*25/30.
    t['BX'] = df['BX_COUNTER']*25.
    t['ORBIT'] = df['ORBIT_CNT']*bx_max*25.
    return t['TDC']+t['BX']+t['ORBIT']

start_time_ns = ns_time(df.iloc[0], bx_max)
stop_time_ns = ns_time(df.iloc[df.shape[0]-1], bx_max)
run_time_ns = stop_time_ns - start_time_ns
print(f"Total run time is {run_time_ns} ns ({run_time_ns/10**9} s)")

#### Exercise 4
Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [ ]:
df['ABS_T'] = (df['TDC_MEAS']*25/30)+(df['BX_COUNTER']*25)+(df['ORBIT_CNT']*bx_max*25)
df.plot

#### Exercise 5
Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [ ]:
df['HEAD'] = np.random.randint(2, size=df.shape[0])

#### Exercise 6
Create a new DataFrame that contains only the rows with HEAD=1

In [ ]:
df_head1 = df[df['HEAD']==1]

#### Exercise 7
Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [ ]:
df_fpga0 = df[df['FPGA']==0]
df_fpga1 = df[df['FPGA']==1]
grouped_0 = df_fpga0.groupby('TDC_CHANNEL', dropna=False)
grouped_1 = df_fpga1.groupby('TDC_CHANNEL', dropna=False)
counts_0 = grouped_0.count()
counts_1 = grouped_1.count()

figure, axis = plt.subplots(1, 2)
figure.set_figwidth(12)
figure.set_figheight(6)

axis[0].plot(counts_0.axes[0], counts_0['FPGA']) 
axis[0].set_title("FPGA 0") 
axis[0].set_xlabel("TDC Channel")
axis[0].set_ylabel("Counts")
axis[1].plot(counts_1.axes[0], counts_1['FPGA'])
axis[1].set_title("FPGA 1") 
axis[1].set_xlabel("TDC Channel")
axis[1].set_ylabel("Counts")
figure.tight_layout(pad=5.0)

plt.show()


#### Exercise 8
Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

In [ ]:
v = grouped_0['HEAD'].count()
v = v.sort_values()
print("FPGA_0")
for i in range(v.shape[0]-1,v.shape[0]-4,-1):
    print(f"TDC channel {v.axes[0][i]} = {v.iloc[i]}")
print()
v1 = grouped_1['HEAD'].count()
v1 = v1.sort_values()
print("FPGA_1")
for i in range(v1.shape[0]-1,v1.shape[0]-4,-1):
    print(f"TDC channel {v1.axes[0][i]} = {v1.iloc[i]}")

#### Exercise 9
Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
n_of_unique_orbits = df.groupby('ORBIT_CNT').count().shape[0]
print(f"Number of unique orbits: {n_of_unique_orbits}")

n_uniq_139 = df[df['TDC_CHANNEL']==139].groupby('ORBIT_CNT').count().shape[0]
print(f"Number of unique orbits with at least one meas from TDC_CHANNEL=139: {n_uniq_139}")